# MAD Dynamique <br>
### STAGE ANCHES <br>
Camille Urban <br>
22/03/2024

In [1]:
# classic shit
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
from scipy.signal import butter, filtfilt

#librosa
import librosa
import librosa.display

# traitement image
from skimage.io import imread
from skimage.transform import rotate
# from im_rotate import im_rotate
import os

import sounddevice as sd
from scipy.io.wavfile import write

In [2]:
%matplotlib
%matplotlib

Using matplotlib backend: <object object at 0x000002BDDB20A360>
Using matplotlib backend: TkAgg


## Paramètres du Banc

In [3]:
# SENSIBILITE CAPTEURS
S_mic_int =  0.0841  # Sensibilité du capteur de pression interne (mV/Pa)
Offset_mic_int = 51.2   # (mV) 
# G_micro_int = 10 V
S_force = 39.2 # mV/N
# Offset_force = 
# G_force = 10 V
# étalonnage de micro exterieur par source étalon 1kHz à 97dB

# FRQ ECHANTILLONNAGE
sr = 10000 
n_fft = 512 # env. 50ms
# hop_length = n_fft / 4  # = 12.5ms (pas de décalage de la fenêtre)
hop_length = n_fft / 4
p_ref = 2e-5

largeur_bande = 100


In [4]:
dossier = 'mes_2024.04.19/'
nom_fichier = 'anche'
na = 1
data = pd.read_table(dossier + nom_fichier + f'{na}',  sep='\t', header=2).values
tps = data[:,0]
force = data[:,1]/S_force
p_int = (data[:,2])/S_mic_int
# print(type(p_int))
p_ext = data[:,3]
# vib = data[:,4]


nombre_anches = 5


# Traitement données

### Normalisation

In [5]:
# NORMALISATION
 
def normalisation(signal):
    # normalized_signal = signal / np.mean(signal)
    normalized_signal = signal / np.max(signal)
    return normalized_signal


plot_fig = False
plt.close('all')
signal = p_int
if plot_fig :
    plt.figure()
    plt.plot(tps, signal, label='Signal initial')
    plt.plot(tps, normalisation(signal), label='Signal normalisé')
    plt.xlabel('Temps (s)')
    plt.ylabel('Amplitude')
    # plt.title('')
    plt.legend()
    plt.show()


In [6]:
# ecoute et enregistrement des signaux en pression 
play_pression_bec = True
play_pression_ext  = True

for na in range (nombre_anches):
    if play_pression_bec:
        p_i = normalisation(p_int)
        audio_signal = np.int16(p_i * 32767)
        # sd.play(audio_signal, sr)
        output_wav = dossier + f'pression_bec_anche{na}.wav'
        write(output_wav, sr, audio_signal)
        
    if play_pression_ext:
        p_e = normalisation(p_ext)
        audio_signal = np.int16(p_e * 32767)
        # sd.play(audio_signal, sr)
        output_wav = dossier + f'pression_ext_anche{na}.wav'
        write(output_wav, sr, audio_signal)

### Fréquence fondamentale 

In [7]:
def freq_fond(signal, sr) :  
    TF = np.fft.fft(signal)
    freqs = np.fft.fftfreq(len(signal), d=1/sr)         # valeurs des frequences du spectre 
    indice_fonda = np.argmax(np.abs(TF[1:])) + 1        # indice de la frequence fondamentale (+1 pour éviter l'offset en 0)
    freq_fonda = np.abs(freqs[indice_fonda])            # fréquence fondamentale
    # print("Fréquence fondamentale :", freq_fonda)
    return freq_fonda


### Filtrer le signale fréquentiellement autour de la fondamentale

In [8]:
# Retourne le signal filtré autour de la fréquence fondamentale
# Prend en entrée le signal et la largeur de la bande passante autour de la fréquence fondamentale
# largeur de la bande frequentielle - freq_bande = 100   
def filtered_sig(signal, sr, largeur_bande):
    freq_fonda = freq_fond(signal, sr)
    
    # Definition de l'ordre du filtre
    ordre_filtre = 4  
    
    # Calcul des fréquences de coupure
    freq_coup_inf = freq_fonda - largeur_bande / 2
    freq_coup_sup = freq_fonda + largeur_bande / 2

    # fréquences de coupure en fréquences normalisées
    frequence_coupure_inf_norm = freq_coup_inf / (sr / 2)
    frequence_coupure_sup_norm = freq_coup_sup / (sr / 2)

    # Création du filtre passe-bande
    b, a = butter(ordre_filtre, [frequence_coupure_inf_norm, frequence_coupure_sup_norm], btype='band')

    # Filtrage du signal
    signal_filtre = filtfilt(b, a, signal)
    return signal_filtre


plot_fig = False
plt.close('all')
signal = p_int
if plot_fig :
    plt.figure()
    plt.plot(tps, normalisation(signal), label='Signal normalisé')
    plt.plot(tps, filtered_sig(normalisation(signal), sr=sr, largeur_bande=largeur_bande), label='Signal normalisé filtré')
    plt.xlabel('Temps (s)')
    plt.ylabel('Amplitude normalisée')
    plt.title('Comparaison entre le signal initial et le signal filtré')
    plt.legend()
    plt.show()

### Enveloppe Spectrale

In [9]:
# Fonction pour calculer l'enveloppe spectrale par convolution avec fenêtre de Hann
def spectral_envelope(signal, window_size): 
    # Calculer le nombre de segments
    num_segments = len(signal) // window_size
    
    # Initialiser l'enveloppe spectrale
    spectral_env = np.zeros(len(signal))
    
    # fenêtrage de Hann sur chaque segment et TF
    for i in range(num_segments):
        start_idx = i * window_size
        end_idx = (i + 1) * window_size
        segment = signal[start_idx:end_idx]
        
        # fenêtrage
        segment *= sp.signal.windows.hann(window_size)
        
        # TF et amplitude
        spectrum = np.fft.fft(segment)
        amplitude = np.abs(spectrum)
        
        spectral_env[start_idx:end_idx] = amplitude
    
    # Lissage par convolution
    spectral_env = np.convolve(spectral_env, np.ones(window_size) / window_size, mode='same')
    
    return spectral_env

plot_fig = False
plt.close('all')
signal = p_int
if plot_fig :
    plt.figure()
    plt.plot(tps, filtered_sig(normalisation(signal), sr=sr, largeur_bande=largeur_bande), label='Signal normalisé filtré')
    plt.plot(tps, spectral_envelope(filtered_sig(normalisation(signal), sr=sr, largeur_bande=largeur_bande), window_size=n_fft), label='Enveloppe spectrale')
    plt.xlabel('Temps (s)')
    plt.ylabel('Amplitude normalisée')
    plt.title('Enveloppe spectrale du signal filtré autour de la fréquence fondamentale')
    plt.legend()
    plt.show()

### Seuillage

In [10]:
# Détermination du seuil de détéction paour chaque l'enveloppe spectrale du signal normalisé et filtré autour de f_0
# On prend le seuillage sur 100ms 

def seuillage(signal, sr, largeur_bande, window_size, facteur_std):
    
    print_seuil = False
    
    signal = normalisation(signal)
    signal = filtered_sig(signal, sr, largeur_bande)
    signal = spectral_envelope(signal, window_size)
    
    longueur_selection = int(0.1*sr) 
    selection = signal[:longueur_selection]
    e_type = np.std(selection)
    seuil = facteur_std*e_type  # seuil = 2*3 idéalement
    if print_seuil:
        print(seuil)
    return seuil

        

### Troncatures

In [11]:
def troncature(signal, sr, largeur_bande, window_size, facteur_std) : 
    signal = normalisation(signal)
    # Signal filtré
    signal_filtre = filtered_sig(signal, sr, largeur_bande)
    # Enveloppe spectrale
    env_spec = spectral_envelope(signal_filtre, window_size)
    # Normalisation de l'enveloppe
    env_spec = spectral_envelope(signal_filtre, window_size) / np.max(spectral_envelope(signal_filtre, window_size))

    e_cum = np.cumsum(env_spec)
    e_cum = normalisation(e_cum) 
    
    # Seuil 
    seuil = seuillage(signal, sr, largeur_bande, window_size, facteur_std)
    
    idx_tr =[]
    seuil_depasse = False
    for i, valeur in enumerate(env_spec):
        if not seuil_depasse and valeur > seuil:
            seuil_depasse = True
            idx_tr.append(i)  # Ajouter le moment où le seuil est dépassé
        elif seuil_depasse and valeur <= seuil:
            seuil_depasse = False
            idx_tr.append(i) 
    
    print(idx_tr)  
    return idx_tr, e_cum
   

In [12]:
plot_env_spec = False
if plot_env_spec:
    plt.plot(tps, filtered_sig(normalisation(signal), sr=sr, largeur_bande=largeur_bande), label='Signal normalisé filtré')
    plt.plot(tps, spectral_envelope(filtered_sig(normalisation(signal), sr=sr, largeur_bande=largeur_bande), window_size=n_fft), label='Enveloppe spectrale')
    plt.grid(True)
    plt.xlabel('Temps (s)')
    plt.ylabel('Energie')
    plt.title('Energie cumulative')
    plt.legend()
    
plt.close('all')   

plot_fig = True
# plt.close('all')
for nbr_anche in range (5):
    #LOAD DATA
    dossier = 'mes_2024.04.19/'
    nom_fichier = 'anche'
    f=20
    data = pd.read_table(dossier + nom_fichier + f'{nbr_anche+1}', sep='\t', header=2).values
    tps = data[:,0]
    force = data[:,1]/S_force
    p_int = (data[:,2])/S_mic_int
    signal = p_int
    
    # Plot seuil
    if plot_fig :
        plt.figure()
        # plt.plot(tps, filtered_sig(normalisation(signal), sr=sr, largeur_bande=largeur_bande), label='Signal normalisé filtré')
        plt.plot(tps, spectral_envelope(filtered_sig(normalisation(signal), sr=sr, largeur_bande=largeur_bande), window_size=n_fft), label='Enveloppe spectrale')
        plt.hlines(seuillage(signal, sr=sr, largeur_bande=largeur_bande, window_size=n_fft, facteur_std=f), tps[0], tps[-1], color='yellow', label='')
        plt.xlabel('Temps')
        plt.ylabel('Amplitude')
        plt.title('Comparaison entre le signal initial et le signal filtré')
        plt.legend()
        plt.show()
        
        # Troncature des signaux
        # tronquer = True
        # if tronquer:
        titres = ['pression croissante', 'pression décroissante']
        idx_tr=[]
        idx_tr, e_cum = troncature(signal=p_int, sr=sr, largeur_bande=100, window_size=n_fft, facteur_std=f)
        
        plt.figure()
        plt.plot(tps[idx_tr[0]:idx_tr[1]], normalisation(p_int[idx_tr[0]:idx_tr[1]]), label='Enveloppe spectrale')
        # plt.plot(tps[idx_tr[0]:idx_tr[1]], normalisation(e_cum[idx_tr[0]:idx_tr[1]]), label='somme cumulée des énergie')
        plt.grid(True)
        plt.xlabel('Temps (s)')
        plt.ylabel('Energie')
        plt.title('Energie cumulative')
        plt.legend()

        plt.figure()
        plt.plot(tps[idx_tr[2]:idx_tr[-1]], normalisation(p_int[idx_tr[2]:idx_tr[-1]]), label='Enveloppe spectrale')
        # plt.plot(tps[idx_tr[2]:idx_tr[-1]], normalisation(e_cum[idx_tr[2]:idx_tr[-1]]), label='somme cumulée des énergie')
        plt.grid(True)
        plt.xlabel('Temps (s)')
        plt.ylabel('Energie')
        plt.title('Energie cumulative')
        plt.legend()

[17162, 25335, 44300, 61196]
[19701, 36108, 45323, 63755]
[21237, 37132, 46859, 59638]
[27915, 41717, 53516, 67339]
[11020, 57078, 57079, 57099, 58124, 80119]


### Spectrogramme des signaux en pression (microphone interne et externe)

In [13]:
# Retourne les sepctrogramme sur la pression interne et externe en dB et les plot
def plot_spectro(signal, n_fft, numero_anche):
    # Spectrogramme
    spectrogram = np.abs(librosa.stft(y=signal, n_fft=n_fft))
    # Mise en dB
    spectrogram_db = librosa.power_to_db(spectrogram**2, ref=p_ref**2)
    
    # Affichages des spectrogrammes
    plt.figure()
    plt.title(f'Sepctrogrammes \nAnche n° {numero_anche}', fontsize=16)
    
    img = librosa.display.specshow(spectrogram_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel')
    cbar = plt.colorbar(img, format='%+2.0f dB')
    cbar.set_label('Amplitude (dB)')
    plt.title('Pression interne au bec')

    plt.tight_layout()
    plt.show()
    return spectrogram_db

plot_spec = True
if plot_spec:
    plt.close('all')      
    for i in range (nombre_anches):
        spectro_db = plot_spectro(p_int, n_fft, i)
